In [ ]:
%load_ext autoreload
%aimport rasterio_helpers
%aimport proj_scope_vars
%autoreload 1
%autosave 300

import importlib 

import pdal
import glob
import json
from rasterio_helpers import *
from proj_scope_vars import *
import rasterio
from rasterio.warp import reproject
import numpy as np

from arosics import DESHIFTER
from arosics import COREG_LOCAL
from geoarray import GeoArray
import os, shutil, glob
import numpy as np
import time
import re
from shapely.geometry import shape
import fiona

In [ ]:
def get_norm_band(boi_nps, nodata):
    scaler = 10000
    
    for band_np in boi_nps:
        amin = np.min(band_np)
        amax = np.max(band_np)
        if band_np is boi_nps[0]:
            band_norm_np = np.where(band_np != nodata, scaler*(band_np-amin)/(amax-amin), nodata)
        else:
            band_norm_np = np.where(np.logical_or((band_np != nodata), (band_norm_np != nodata)), 
                            (band_norm_np + scaler*((band_np-amin)/(amax-amin))), 
                            nodata)
    return band_norm_np

In [ ]:
start_time = time.time()  
plscope_tiffs = glob.glob(f'{pscope_study_dir}/*.tif')
workdir = f'{pscope_intraband_aligned_dir}/work'
ref_band = 'red'

print(plscope_tiffs)

for plscope_tiff in plscope_tiffs:
    aligned_band_nps = {}
    band_descr = {}
    
    proj_name = os.path.splitext(os.path.basename(plscope_tiff))[0]
    proj_dir = f'{workdir}/{proj_name}'
    aligned_tiff = f'{pscope_intraband_aligned_dir}/{proj_name}_aligned.tif'
    
    print(f'Project Name: {proj_name}')
    print(f'Project Directory: {proj_dir}')
    print(f'Output Aligned Tiff: {aligned_tiff}')
    
    try:
        os.makedirs(proj_dir)
    except FileExistsError:
        shutil.rmtree(proj_dir)
        os.makedirs(proj_dir)

    band_gas = {}
        
    file_ga = GeoArray(plscope_tiff, nodata=0, q=True)
    cols = file_ga.columns
    rows = file_ga.rows
    
    band_descr = file_ga.bandnames.items()

    print(band_descr, len(band_descr))
    
    for band_name, band_num in band_descr:
        band_a = file_ga[:rows, :cols, band_num]
        band_gas[band_name] = GeoArray(band_a, 
                           geotransform=file_ga.geotransform,  
                           projection=file_ga.projection,
                           nodata=0,
                           bandnames=[band_name],
                           basename = f'{proj_name}_{band_name}'
                          )
#         outfile = band_dirname + '/' + basename + '.tif'
#         band_gas[band_name].save(outfile, 'GTiff')

    profile={}
    nodata = 0.0
    print(band_gas)
    ref_ga = band_gas[ref_band]
    
    for band_name, band_ga in band_gas.items():
        band_proj_name = f'{proj_name}_{band_name}_{ref_band}'
        band_proj_path = f'{proj_dir}/{band_proj_name}'
        print(f'Base Path: {base_path}')
        
        try:
            os.makedirs(band_proj_path)
        except FileExistsError:
            shutil.rmtree(band_proj_path)
            os.makedirs(band_proj_path)
        
        out_tiff  = f'{band_proj_path}.tif'
        
        if band_name == ref_band:
            band_gas[band_name].save(out_tiff , 'GTiff') 
            with rasterio.open(out_tiff) as aligned_band_ds:
                aligned_band_nps[band_name] = aligned_band_ds.read()
                profile = aligned_band_ds.profile
                nodata = aligned_band_ds.nodata
                
        else:
            print("processing band ", band_name) 

            vect_fig_file = f'{band_proj_path}/{band_proj_name}_vect.png'
            ang_fig_file  = f'{band_proj_path}/{band_proj_name}_ang.png'
            mag_fig_file  = f'{band_proj_path}/{band_proj_name}_mag.png'
            tiepoint_file = f'{band_proj_path}/{band_proj_name}.shp'
            tiepoint_table_file = f'{band_proj_path}/{band_proj_name}.csv'
            
            print("out_tiff ", out_tiff)             
            print("vect_fig_file ", vect_fig_file) 
            print("ang_fig_file ", ang_fig_file) 
            print("mag_fig_file ", mag_fig_file) 
            print("tiepoint_file ", tiepoint_table_file) 
            print("tiepoint_table_file ", tiepoint_table_file) 
            
            kwargs = {
                'grid_res'     : 100,
                'window_size'  : (256,256),
                'path_out'     : out_tiff,
                'fmt_out'      : 'GTIFF',
                'tieP_filter_level' : 2,       
                'min_reliability' : 30,              
                'CPUs'         : 6,
                'nodata'       : (0,0),
            }

            CRL = COREG_LOCAL(ref_ga, band_gas[band_name], **kwargs)

            print("Generating: ", tiepoint_table_file)                
            CRL.CoRegPoints_table.to_csv(tiepoint_table_file)

            #Some of these outputs are undocument, but cool.
            #The cool vector plot
            print("Generating: ", ang_fig_file)        
            CRL.view_CoRegPoints(figsize=(20,20), savefigPath=vect_fig_file, shapes2plot='vectors', vector_scale=15)
            #Plotting cool stuff

            print("Generating: ", vect_fig_file)
            CRL.view_CoRegPoints(figsize=(20,20), savefigPath=ang_fig_file,  attribute2plot='ANGLE')

            print("Generating: ", mag_fig_file)        
            CRL.view_CoRegPoints(figsize=(20,20), savefigPath=mag_fig_file )

            print("Generating: ", out_tiff)                
            CRL.correct_shifts()
        
        with rasterio.open(out_tiff) as aligned_band_ds:
            aligned_band_nps[band_name] = aligned_band_ds.read()
            
    
    band_descr = dict(file_ga.bandnames.items())
    boi_names = ['green', 'yellow', 'red', 'rededge', 'nir']
    boi = [aligned_band_nps[boi_name] for boi_name in boi_names]

    aligned_band_nps['normalized bands of interest'] = get_norm_band(boi, nodata)
    band_descr['normalized bands of interest'] = len(aligned_band_nps)-1

    profile.update( { 'count': len(aligned_band_nps) } )

    with rasterio.open(aligned_tiff, 'w', **profile) as dst:
        for band_name, band_idx  in band_descr.items():
            band = band_idx+1
            print(band, band_name, aligned_band_nps[band_name][0].shape)

            dst.write(aligned_band_nps[band_name][0], band)
            dst.set_band_description(band, band_name)
    
end_time = time.time()     
print("Done.  Elapsed Time:", start_time-end_time)

In [ ]:
for band,a in aligned_band_nps.items():
    print(a.shape)

In [ ]:
band_descr